In [1]:
import json
import glob, pickle

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima, FakeBelem

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel
plt.style.use({'figure.facecolor':'white'})

In [2]:
backend_lima = FakeLima()
backend_belem = FakeBelem()
backend_montreal = FakeMontreal()
properties_lima = get_backend_properties_v1(backend_lima)
properties_belem = get_backend_properties_v1(backend_belem)
properties_montreal = get_backend_properties_v1(backend_montreal)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_lima = AerSimulator.from_backend(backend_lima) # Noisy
backend_noisy_belem = AerSimulator.from_backend(backend_belem) # Noisy
backend_noisy_montreal = AerSimulator.from_backend(backend_montreal) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy_lima = {'shots': 10000, 'backend': backend_noisy_lima, 'name': 'noisy_lima'}
run_config_noisy_belem = {'shots': 10000, 'backend': backend_noisy_belem, 'name': 'noisy_belem'}
run_config_noisy_montreal = {'shots': 10000, 'backend': backend_noisy_montreal, 'name': 'noisy_montreal'}

BATCH_SIZE = 32

# Sweep Fintuning

In [3]:
model_path = './model/finetuning/train_fakelima.pth'

model = ExpValCircuitGraphModel(
    num_node_features=22,
    hidden_channels=15,
    exp_value_size=1
)
model.load_state_dict(torch.load(model_path))

criterion = torch.nn.MSELoss()

optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer,
                              'min',
                              factor=0.1,
                              patience=15,
                              verbose=True,
                              min_lr=0.00001)

finetune_paths = [
    f'./data/circ_parsed_pyg_data/finetune/fakemontreal_depth{i}.json' for i in range(1, 10)
]

test_paths = [
    f'./data/circ_parsed_pyg_data/val/fakemontreal_depth{i}.json' for i in range(1, 10)
]

finetune_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        finetune_paths,
        num_samples=300
    ),
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_loader = DataLoader(
    CircuitGraphExpValMitigationDataset(
        test_paths,
    ),
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(sum(p.numel() for p in model.parameters() if p.requires_grad), len(finetune_loader) * BATCH_SIZE, len(test_loader) * BATCH_SIZE)

In [7]:
min_valid_loss = np.inf

finetune_losses = []
test_losses = []

N_EPOCHS = 50

progress = tqdm_notebook(range(N_EPOCHS), desc='Model training', leave=True)
for epoch in progress:
    finetune_loss = 0.0
    model.train()
    for i, data in enumerate(finetune_loader):
        optimizer.zero_grad()

        out = model(
            data.noisy_0,
            data.observable,
            data.circuit_depth,
            data.x,
            data.edge_index,
            data.batch
        )
        loss = criterion(out, torch.squeeze(data.y, 1))

        finetune_loss += loss.item()

        loss.backward()
        optimizer.step()

    test_loss = 0.0
    model.eval()
    for i, data in enumerate(test_loader):
        out = model(
            data.noisy_0,
            data.observable,
            data.circuit_depth,
            data.x,
            data.edge_index,
            data.batch)
        loss = criterion(out, torch.squeeze(data.y, 1))

        test_loss += loss.item()

    scheduler.step(test_loss)

    if epoch >= 1:
        finetune_losses.append(finetune_loss / len(finetune_loader))
        test_losses.append(test_loss / len(test_loader))

        progress.set_description(f"{round(finetune_losses[-1], 5)}, {round(test_losses[-1], 5)}")
        progress.refresh()



plt.style.use({'figure.facecolor':'white'})
plt.plot(finetune_losses, label="train_loss")
plt.plot(test_losses, label="val_loss")
plt.title('Finetune and test on fakemonreal')
plt.yscale('log')
plt.ylim([0.0001, 0.5])

plt.legend()
plt.show()

model_path = './model/finetuning/finetune_fakemontreal.pth'

torch.save(model.state_dict(), model_path)

to_save = {'finetune_losses': finetune_losses, 'test_losses': test_losses}
with open('.'+model_path.split('.')[1]+'.pk', 'wb') as handle:
    pickle.dump(to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)

Model training:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 00029: reducing learning rate of group 0 to 1.0000e-04.
Epoch 00045: reducing learning rate of group 0 to 1.0000e-05.


In [18]:
# with open('./model/finetuning/finetune_fakemontreal.pk', 'rb') as in_file:
#     curves = pickle.load(in_file)
#
# finetune_losses, test_losses = curves['finetune_losses'], curves['test_losses']